In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np
from keras.utils import np_utils

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기

# gcc4_0~3
gcc_version = 'gcc4'
gcc4_0_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_0_32.csv", index_col=0)
gcc4_1_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_1_32.csv", index_col=0)
gcc4_2_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_2_32.csv", index_col=0)
gcc4_3_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_3_32.csv", index_col=0)

# 형태 출력
print(gcc4_0_32.shape, gcc4_1_32.shape, gcc4_2_32.shape, gcc4_3_32.shape) # train

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc4_0_32.reset_index(inplace=True, drop=True)
gcc4_1_32.reset_index(inplace=True, drop=True)
gcc4_2_32.reset_index(inplace=True, drop=True)
gcc4_3_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
gcc4_0_32.head()

Using TensorFlow backend.


(2156208, 2) (1724990, 2) (1773289, 2) (2224199, 2)
reset_index 완료


,bin,label
0,55,1
1,89,0
2,e5,0
3,53,0
4,83,0


In [2]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc4_0_32['bin'].unique()),len(gcc4_1_32['bin'].unique()))
print(len(gcc4_2_32['bin'].unique()),len(gcc4_3_32['bin'].unique()))

256 256
256 256


## 원 핫 인코딩 (16진수 256 = 00 ~ ff -> 000000... 1 ...0000) 

In [3]:
# (3) get_dummies 변환 - onehot encoding

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc4_0_32_onehot = pd.get_dummies(gcc4_0_32)
gcc4_1_32_onehot = pd.get_dummies(gcc4_1_32)
gcc4_2_32_onehot = pd.get_dummies(gcc4_2_32)
gcc4_3_32_onehot = pd.get_dummies(gcc4_3_32)
print('원핫인코딩완료')

print(gcc4_0_32_onehot.shape, gcc4_1_32_onehot.shape, gcc4_2_32_onehot.shape, gcc4_3_32_onehot.shape)

원핫인코딩완료
(2156208, 257) (1724990, 257) (1773289, 257) (2224199, 257)


In [4]:
# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함

# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc4_0_32_onehot['label'].value_counts(), '\n', gcc4_1_32_onehot['label'].value_counts())
print(gcc4_2_32_onehot['label'].value_counts(), '\n', gcc4_3_32_onehot['label'].value_counts())

0    2151901
1       4307
Name: label, dtype: int64 
 0    1721737
1       3253
Name: label, dtype: int64
0    1770091
1       3198
Name: label, dtype: int64 
 0    2221208
1       2991
Name: label, dtype: int64


In [5]:
# 전체 행렬에 대한 value_counts()
gcc4_0_32['bin'].value_counts()

00    322630
ff    186095
8b    155808
45    113507
89    100993
       ...  
a9       269
9e       266
b1       252
96       252
a6       239
Name: bin, Length: 256, dtype: int64

## 함수의 시작 '1' 기준 앞뒤 10gram 잘라내기 (O0 ~ O4)

In [6]:
# (5-1) gcc4 - 함수의 시작 기준 앞뒤 10gram
######################## gcc4_0_32
idx0 = gcc4_0_32_onehot[gcc4_0_32_onehot['label']==1].index  # 407, 474 ...
ls0 = list(idx0)

# 최종 뽑을 행에 대한 index
ls_idx0 = [] 
left_idx, right_idx = -10, 11 # 앞뒤 10개씩

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx0.extend(list(idx0 + k)) # index 형이라서 가능

ls_idx0 = list(set(ls_idx0))    
# print(len(ls_idx0))

ls_idx0.sort() # 인덱스 정렬
ls_idx0 = list(filter(lambda x: x>0, ls_idx0)) # 0이상인 값들만 
ls_idx0 = list(filter(lambda x: x<len(gcc4_0_32_onehot), ls_idx0))

print('gcc4_0_32', len(ls_idx0))

# loc 로 수정필요
gcc4_0_32_onehot_30gram = gcc4_0_32_onehot.loc[ls_idx0,:].copy()

gcc4_0_32 87572


In [7]:
# (5-2) gcc4 - 함수의 시작 기준 앞뒤 10gram
######################## gcc4_1_32
idx1 = gcc4_1_32_onehot[gcc4_1_32_onehot['label']==1].index  # 407, 474 ...
ls1 = list(idx1)

# 최종 뽑을 행에 대한 index
ls_idx1 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx1.extend(list(idx1 + k)) # index 형이라서 가능

ls_idx1 = list(set(ls_idx1))    
# print(len(ls_idx0))

ls_idx1.sort() # 인덱스 정렬
ls_idx1 = list(filter(lambda x: x>0, ls_idx1)) # 0이상인 값들만 
ls_idx1 = list(filter(lambda x: x<len(gcc4_1_32_onehot), ls_idx1))

print('gcc4_1_32', len(ls_idx1))

# loc 로 수정필요
gcc4_1_32_onehot_30gram = gcc4_1_32_onehot.loc[ls_idx1,:].copy()


gcc4_1_32 65777


In [8]:
# (5-3) gcc4 - 함수의 시작 기준 앞뒤 10gram
######################## gcc4_2_32
idx2 = gcc4_2_32_onehot[gcc4_2_32_onehot['label']==1].index  # 407, 474 ...
ls2 = list(idx2)

# 최종 뽑을 행에 대한 index
ls_idx2 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx2.extend(list(idx2 + k)) # index 형이라서 가능

ls_idx2 = list(set(ls_idx2))    
# print(len(ls_idx0))

ls_idx2.sort() # 인덱스 정렬
ls_idx2 = list(filter(lambda x: x>0, ls_idx2)) # 0이상인 값들만 
ls_idx2 = list(filter(lambda x: x<len(gcc4_2_32_onehot), ls_idx2))

print('gcc4_2_32', len(ls_idx2))

# loc 로 수정필요
gcc4_2_32_onehot_30gram = gcc4_2_32_onehot.loc[ls_idx2,:].copy()


gcc4_2_32 65474


In [9]:

# (5-3) gcc4 - 함수의 시작 기준 앞뒤 10gram
######################## gcc4_3_32
idx3 = gcc4_3_32_onehot[gcc4_3_32_onehot['label']==1].index  # 407, 474 ...
ls3 = list(idx3)

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능

ls_idx3 = list(set(ls_idx3))    
# print(len(ls_idx0))

ls_idx3.sort() # 인덱스 정렬
ls_idx3 = list(filter(lambda x: x>0, ls_idx3)) # 0이상인 값들만 
ls_idx3 = list(filter(lambda x: x<len(gcc4_3_32_onehot), ls_idx3))

print('gcc4_3_32', len(ls_idx3))

# loc 로 수정필요
gcc4_3_32_onehot_30gram = gcc4_3_32_onehot.loc[ls_idx3,:].copy()


gcc4_3_32 61356


## Train, Test Label split

In [10]:
# (7) 훈련데이터, 훈련라벨 분리 012

# 훈련 데이터, 훈련 라벨
x_gcc4_0_32 = gcc4_0_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc4_0_32 = gcc4_0_32_onehot_30gram['label'].to_numpy()

x_gcc4_1_32 = gcc4_1_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc4_1_32 = gcc4_1_32_onehot_30gram['label'].to_numpy()

x_gcc4_2_32 = gcc4_2_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc4_2_32 = gcc4_2_32_onehot_30gram['label'].to_numpy()

# 테스트 데이터, 테스트 라벨
x_gcc4_3_32 = gcc4_3_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc4_3_32 = gcc4_3_32_onehot_30gram['label'].to_numpy()

# train
print(x_gcc4_0_32.shape, y_gcc4_0_32.shape)
print(x_gcc4_1_32.shape, y_gcc4_1_32.shape)
print(x_gcc4_2_32.shape, y_gcc4_2_32.shape)

# test
print(x_gcc4_3_32.shape, y_gcc4_3_32.shape)

(87572, 256) (87572,)
(65777, 256) (65777,)
(65474, 256) (65474,)
(61356, 256) (61356,)


## Reshape

In [11]:
# (8) 훈련데이터, 훈련라벨 reshape 작업 => # ex - (605, 1, 256)

## gcc6_0_32
x_gcc4_0_32 = x_gcc4_0_32.reshape(x_gcc4_0_32.shape[0], 1, x_gcc4_0_32.shape[1])
y_gcc4_0_32 = y_gcc4_0_32.reshape(y_gcc4_0_32.shape[0], 1, 1)

## gcc6_1_32
x_gcc4_1_32 = x_gcc4_1_32.reshape(x_gcc4_1_32.shape[0], 1, x_gcc4_1_32.shape[1])
y_gcc4_1_32 = y_gcc4_1_32.reshape(y_gcc4_1_32.shape[0], 1, 1)

## gcc6_2_32
x_gcc4_2_32 = x_gcc4_2_32.reshape(x_gcc4_2_32.shape[0], 1, x_gcc4_2_32.shape[1])
y_gcc4_2_32 = y_gcc4_2_32.reshape(y_gcc4_2_32.shape[0], 1, 1)

### test data
## gcc6_3_32
x_gcc4_3_32 = x_gcc4_3_32.reshape(x_gcc4_3_32.shape[0], 1, x_gcc4_3_32.shape[1])
y_gcc4_3_32 = y_gcc4_3_32.reshape(y_gcc4_3_32.shape[0], 1, 1)

print(x_gcc4_0_32.shape, y_gcc4_0_32.shape)
print(x_gcc4_1_32.shape, y_gcc4_1_32.shape)
print(x_gcc4_2_32.shape, y_gcc4_2_32.shape)

## test data
print(x_gcc4_3_32.shape, y_gcc4_3_32.shape)

(87572, 1, 256) (87572, 1, 1)
(65777, 1, 256) (65777, 1, 1)
(65474, 1, 256) (65474, 1, 1)
(61356, 1, 256) (61356, 1, 1)


## 전체 Model O0 ~ O4 다 합치기 np.concatenate

In [12]:
# (9) 최종 훈련데이터, 훈련라벨 concatenate 작업
# 0123 / 3
x_gcc4_0123_32 = np.concatenate((x_gcc4_0_32, x_gcc4_1_32, x_gcc4_2_32, x_gcc4_3_32), axis=0)
y_gcc4_0123_32 = np.concatenate((y_gcc4_0_32, y_gcc4_1_32, y_gcc4_2_32, y_gcc4_3_32), axis=0)
print(x_gcc4_0123_32.shape, y_gcc4_0123_32.shape)

(280179, 1, 256) (280179, 1, 1)


## Modeling

In [13]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = Input(batch_shape=(None, 1, 256)) 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model1 = Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 256)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1, 32)             34944     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________


## 교차 검증 (10 - Kfold) & 평가지표

In [14]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_gcc4_0123_32, y_gcc4_0123_32):
    print('======Training stage======')
    model1.fit(x_gcc4_0123_32[train],
               y_gcc4_0123_32[train],
               epochs = 3,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_gcc4_0123_32[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc4_0123_32[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

======Training stage======
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
252161/252161 [==============================] - 17s 66us/step - loss: 0.0929 - accuracy: 0.9743
Epoch 2/3
252161/252161 [==============================] - 17s 66us/step - loss: 0.0806 - accuracy: 0.9757
Epoch 3/3
252161/252161 [==============================] - 17s 69us/step - loss: 0.0806 - accuracy: 0.9757


accuracy_score 0.9750874437861374
recall_score 0.5220216606498195
precision_score 0.9525691699604744
f1_score 0.6744402985074627
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 22s 87us/step - loss: 0.0813 - accuracy: 0.9754
Epoch 2/3
252161/252161 [==============================] - 19s 74us/step - loss: 0.0815 - accuracy: 0.9754
Epoch 3/3
252161/252161 [==============================] - 20s 79us/step - loss: 0.0810 - accuracy: 0.9754


accuracy_score 0.9782639731601114
recall_score 0.558667676003028
precision_score 0.9659685863874345
f1_score 0.7079136690647482
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 18s 70us/step - loss: 0.0806 - accuracy: 0.9757
Epoch 2/3
252161/252161 [==============================] - 22s 87us/step - loss: 0.0808 - accuracy: 0.9757
Epoch 3/3
252161/252161 [==============================] - 19s 74us/step - loss: 0.0811 - accuracy: 0.9757


accuracy_score 0.975587122564066
recall_score 0.5388489208633094
precision_score 0.9457070707070707
f1_score 0.6865261228230981
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 19s 75us/step - loss: 0.0808 - accuracy: 0.9758
Epoch 2/3
252161/252161 [==============================] - 18s 71us/step - loss: 0.0808 - accuracy: 0.9758
Epoch 3/3
252161/252161 [==============================] - 21s 82us/step - loss: 0.0802 - accuracy: 0.9758


accuracy_score 0.9749089870797345
recall_score 0.521551724137931
precision_score 0.9515072083879423
f1_score 0.6737819025522042
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 19s 75us/step - loss: 0.0803 - accuracy: 0.9758
Epoch 2/3
252161/252161 [==============================] - 19s 74us/step - loss: 0.0808 - accuracy: 0.9758
Epoch 3/3
252161/252161 [==============================] - 18s 73us/step - loss: 0.0807 - accuracy: 0.9758


...

accuracy_score 0.9751231351274181
recall_score 0.5253256150506512
precision_score 0.9465449804432855
f1_score 0.6756630991158677
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 19s 75us/step - loss: 0.0805 - accuracy: 0.9758
Epoch 2/3
252161/252161 [==============================] - 18s 72us/step - loss: 0.0808 - accuracy: 0.9758
Epoch 3/3
252161/252161 [==============================] - 18s 71us/step - loss: 0.0808 - accuracy: 0.9758


accuracy_score 0.9748376043971733
recall_score 0.5271317829457365
precision_score 0.9565217391304348
f1_score 0.6796910495229441
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 18s 70us/step - loss: 0.0803 - accuracy: 0.9758
Epoch 2/3
252161/252161 [==============================] - 18s 70us/step - loss: 0.0802 - accuracy: 0.9758
Epoch 3/3
252161/252161 [==============================] - 18s 70us/step - loss: 0.0803 - accuracy: 0.9758


accuracy_score 0.9746234563494897
recall_score 0.508400292184076
precision_score 0.9482288828337875
f1_score 0.6619115549215406
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 18s 69us/step - loss: 0.0812 - accuracy: 0.9754
Epoch 2/3
252161/252161 [==============================] - 19s 73us/step - loss: 0.0809 - accuracy: 0.9754
Epoch 3/3
252161/252161 [==============================] - 19s 76us/step - loss: 0.0809 - accuracy: 0.9754


accuracy_score 0.9781568991362696
recall_score 0.5456656346749226
precision_score 0.9657534246575342
f1_score 0.6973293768545994
======Training stage======
Epoch 1/3
252162/252162 [==============================] - 19s 76us/step - loss: 0.0801 - accuracy: 0.9757
Epoch 2/3
252162/252162 [==============================] - 19s 74us/step - loss: 0.0804 - accuracy: 0.9757
Epoch 3/3
252162/252162 [==============================] - 19s 75us/step - loss: 0.0802 - accuracy: 0.9757


accuracy_score 0.9753720955134383
recall_score 0.5472358292512246
precision_score 0.9478787878787879
f1_score 0.6938775510204082

K-fold cross validation Accuracy: [0.9750874437861374, 0.9782639731601114, 0.975587122564066, 0.9749089870797345, 0.974766221714612, 0.9751231351274181, 0.9748376043971733, 0.9746234563494897, 0.9781568991362696, 0.9753720955134383]

K-fold cross validation Recall: [0.5220216606498195, 0.558667676003028, 0.5388489208633094, 0.521551724137931, 0.5087847730600292, 0.5253256150506512, 0.5271317829457365, 0.508400292184076, 0.5456656346749226, 0.5472358292512246]

K-fold cross validation Precision: [0.9525691699604744, 0.9659685863874345, 0.9457070707070707, 0.9515072083879423, 0.9507523939808481, 0.9465449804432855, 0.9565217391304348, 0.9482288828337875, 0.9657534246575342, 0.9478787878787879]

K-fold cross validation F1-Score: [0.6744402985074627, 0.7079136690647482, 0.6865261228230981, 0.6737819025522042, 0.6628516928946113, 0.6756630991158677, 0.67969104952

In [18]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9756726740097349
10-Fold Cross_validation. Recall : 0.529789968837877
10-Fold Cross_validation. Precision : 0.969602268630877
10-Fold Cross_validation. F1-Score : 0.6323657927482637
